In [4]:
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import json
import psycopg2

In [10]:

# Load all credentials 
with open(rf'C:\Users\gurpr\OneDrive\Documents\New Projects\Weather and Pollution\Data_ETL\config.json') as config_file:
    config = json.load(config_file)

db_name = config['db_name']
db_user = config['user']
db_pass = config['password']
db_host = 'localhost'
db_port = config['port']

# Connect to the PostgreSQL database
conn = psycopg2.connect(dbname=db_name, user=db_user, password=db_pass, host=db_host, port=db_port)
cursor = conn.cursor()


def sql_to_dataframe(conn, query):
   '''
   Import data from a PostgreSQL database using a SELECT query 
   '''
   cursor = conn.cursor()
   cursor.execute(query)
   tuples_list = cursor.fetchall()
   cursor.close()
   column_names = [desc[0] for desc in cursor.description]
   df = pd.DataFrame(tuples_list, columns= column_names)
   
   return df

# Load all the data from the tables
cities_query = 'SELECT * FROM cities'
Cities_df = sql_to_dataframe(conn , cities_query)

weather_query = 'SELECT * FROM weather'
Weather_df = sql_to_dataframe(conn , weather_query)

pollution_query = 'SELECT * FROM pollution'
Pollution_df = sql_to_dataframe(conn , pollution_query)


#Change resolution of date_time to hour
Weather_df['date_time_hour'] = Weather_df['date_time'].dt.ceil(freq='h')  
Pollution_df['date_time_hour'] = Pollution_df['date_time'].dt.ceil(freq='h')  

#Merge dfs
merged_df = Weather_df.merge(Pollution_df, how='inner', left_on=['city_id', 'date_time_hour'], right_on=['city_id', 'date_time_hour'])

Exploratory Data Analysis 

In [25]:
merged_df.info()
merged_df.dropna(subset=['temperature','pm2_5','pm10'],inplace = True)
merged_df.info()




<class 'pandas.core.frame.DataFrame'>
Index: 716 entries, 0 to 749
Data columns (total 32 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   weather_id           716 non-null    int64         
 1   city_id              716 non-null    int64         
 2   date_time_x          716 non-null    datetime64[ns]
 3   local_time_x         716 non-null    datetime64[ns]
 4   temperature          716 non-null    float64       
 5   feels_like           710 non-null    float64       
 6   temp_min             704 non-null    float64       
 7   temp_max             709 non-null    float64       
 8   pressure             716 non-null    int64         
 9   humidity             716 non-null    int64         
 10  visibility           716 non-null    int64         
 11  wind_speed           645 non-null    float64       
 12  wind_deg             716 non-null    int64         
 13  clouds_all           716 non-null    int